<a href="https://colab.research.google.com/github/srinivasanibmbangalore/AnalyticsVidhyaCourseOnDL/blob/master/Keras_Dropout_BatchNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
train_csv_path='/storage/Dropout_EarlyStopping/train_man_woman.csv'
test_csv_path='/storage/Dropout_EarlyStopping/test_fkwGUNG1.csv'
images_path='/storage/Dropout_EarlyStopping/images1/'
output_path='/storage/Dropout_EarlyStopping/'
root_path='/storage/Dropout_EarlyStopping/'

In [ ]:
zip_file_name=root_path+'/images1.zip'
with ZipFile(zip_file_name, 'r') as zip:
    zip.extractall() 
    print('Done!') 

In [ ]:
from zipfile import ZipFile 
import argparse
import os
import fnmatch
import shutil
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from psutil import virtual_memory


from keras import backend as K 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation,InputLayer, Conv2D, MaxPool2D, Flatten,Dropout
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers import GlobalMaxPool2D

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.utils import to_categorical

from keras.preprocessing.image import load_img, img_to_array,array_to_img

In [ ]:
num_files=0
clmn_names=['image_name','imgDirLocation']
imgDF = pd.DataFrame(columns=clmn_names)
imgDF.describe()

In [ ]:
seed = 42
rng = np.random.RandomState(seed)
tr_data = pd.read_csv(train_csv_path)
tr_data.describe()

In [ ]:
tr_data.head()

In [ ]:
#Count the number of files in the directory
a=len(fnmatch.filter(os.listdir(images_path),'*.jpg'))
print(a)

In [ ]:
%%time
X=[]
i=0
for index,row in tqdm(tr_data.iterrows()):
    file_path=images_path+'/'+row['image_names']
    img=load_img(file_path)
    img_arr = img_to_array(img)
    X.append(img_arr)
    i+=1
X_a = np.array(X)
X_a.shape

In [ ]:
used_ramgb=virtual_memory().available/1e9
print("Available Memory is " + str(used_ramgb) + " gb")

In [ ]:
del X

In [ ]:
Y_a=tr_data['class'].to_numpy()
Y_a.shape

In [ ]:
X_a.shape

In [ ]:
X_train, X_valid, y_train, y_valid=train_test_split(X_a,Y_a,test_size=0.3, random_state=seed)

In [ ]:
del X_a

In [ ]:
y_valid=np.expand_dims(y_valid,axis=1)
y_train=np.expand_dims(y_train,axis=1)

In [ ]:
y_valid.shape

In [ ]:
y_train.shape

In [ ]:
X_train = np.reshape(X_train,(-1,224*224*3))
X_valid = np.reshape(X_valid,(-1,224*224*3))
X_train /= 255
X_valid /= 255

In [ ]:
X_train.shape
X_valid.shape

In [ ]:
model = Sequential()
model.add(Dense(100, input_dim=224*224*3, activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(50,activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
model.summary()

In [ ]:
%%time
history = model.fit(X_train,y_train, epochs=20, batch_size=32, verbose=2, validation_data=(X_valid,y_valid))

In [ ]:
def plotLosses(history):  
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
plotLosses(history)

In [ ]:
def plotAccuracy(history):  
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
print(history.history.keys())

In [ ]:
plotAccuracy(history)

In [ ]:
#### Reduce the image size and reload
%%time
X=[]
i=0
for index,row in tqdm(tr_data.iterrows()):
    file_path=images_path+'/'+row['image_names']
    img=load_img(file_path,target_size=(112,112))
    img_arr = img_to_array(img)
    X.append(img_arr)
    i+=1
X_a = np.array(X)
X_a.shape

In [ ]:
Y_a=tr_data['class'].to_numpy()
Y_a.shape

In [ ]:
X_train, X_valid, y_train, y_valid=train_test_split(X_a,Y_a,test_size=0.3, random_state=seed)

In [ ]:
y_valid=np.expand_dims(y_valid,axis=1)
y_train=np.expand_dims(y_train,axis=1)
X_train = np.reshape(X_train,(-1,112*112*3))
X_valid = np.reshape(X_valid,(-1,112*112*3))
X_train /= 255
X_valid /= 255
X_train.shape


In [ ]:
model = Sequential()
model.add(Dense(100, input_dim=112*112*3, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="sgd", metrics=['accuracy'])
model.summary()

In [ ]:
%%time
history = model.fit(X_train,y_train, epochs=50, verbose=2, validation_data=(X_valid,y_valid))

In [ ]:
plotLosses(history)

In [ ]:
plotAccuracy(history)

In [ ]:
## Let us introduce some drop-outs to check validation accuracy can be further refined.
model = Sequential()
model.add(Dense(100, input_dim=112*112*3, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="sgd", metrics=['accuracy'])
model.summary()

In [ ]:
%%time
history = model.fit(X_train,y_train, epochs=50, verbose=2, validation_data=(X_valid,y_valid))

In [ ]:
plotAccuracy(history)

In [ ]:
model = Sequential()
model.add(Dense(100, input_dim=112*112*3, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(50,activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="sgd", metrics=['accuracy'])
model.summary()

In [ ]:
%%time
history = model.fit(X_train,y_train, epochs=50, verbose=2, validation_data=(X_valid,y_valid))

In [ ]:
plotAccuracy(history)

In [ ]:
plotLosses(history)